# Import

In [2]:
import os
import sys
from datetime import datetime, timedelta
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import re
import string
from tqdm import tqdm

# Constants

In [7]:
CHROME_DRIVER_PATH = "c:\\Users\\smazzanti\\GitProjects\\personal\\tds-crawler\\chromedriver\\chromedriver.exe"

# Utils

In [8]:
def get_driver(executable_path, silent=True):
    options = webdriver.chrome.options.Options()
    options.headless = silent
    driver = webdriver.Chrome(executable_path=executable_path, chrome_options=options)
    return driver

def clean_url(user_url):
    return user_url.split('?source=')[0].rstrip("/")

# Retrieve info

In [12]:
# retrieve main info of all articles published by towards data science between start_date and end_date (included)
start_date = datetime(2022, 10, 21).date()
end_date = datetime(2022, 10, 22).date()
articles_info = list()

In [13]:
driver = get_driver(executable_path=CHROME_DRIVER_PATH)

article_dates = [start_date + timedelta(days=days) for days in range((end_date - start_date).days + 1)]

for article_date in tqdm(article_dates):
    archive_url = "https://towardsdatascience.com/archive/" + datetime.strftime(article_date, "%Y/%m/%d")
    driver.get(archive_url)

    if driver.current_url != archive_url: # when a page is missing, the driver is sent to the month page
        continue

    articles = driver.find_elements_by_xpath("/html/body/div[1]/div[2]/div/div[3]/div[1]/div[2]/div[.]")

    for article in articles:
        try:
            article_title = article.find_element_by_tag_name("h3").text
        except:
            article_title = article.find_element_by_tag_name("h2").text
        article_url = clean_url(article.find_element_by_xpath("./div/div/div[2]/a").get_attribute("href"))         
        user = article.find_element_by_xpath("./div/div/div[1]/div/div/div[2]/a[1]")
        user_url = clean_url(user.get_attribute("href"))
        user_name = user.text
        reading_time = int(article.find_element_by_class_name("readingTime").get_attribute("title").split(" ")[0])
        n_claps = int(eval(article.find_element_by_xpath("./div/div/div[4]/div[1]/div/span").text.zfill(1).replace("K", "*1000")))

        articles_info.append({
            "ref_date": datetime.now().date(),
            "article_url": article_url,
            "article_title": article_title,
            "article_date": article_date,
            "user_url": user_url,
            "user_name": user_name,
            "reading_time": reading_time,
            "n_claps": n_claps
        })

driver.quit()

C:\Users\smazzanti\PycharmProjects\mrmr\venv\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.70it/s]


In [14]:
articles_info

[]

In [26]:
df_articles_info = pd.DataFrame(articles_info)
df_articles_info.shape

(6635, 8)

In [31]:
df_articles_info = df_articles_info.drop_duplicates()
df_articles_info.shape

(6617, 8)

In [32]:
df_articles_info.nunique()

ref_date            1
article_url      6617
article_title    6588
article_date      363
user_url         1883
user_name        1883
reading_time       40
n_claps           626
dtype: int64

In [33]:
df_articles_info.isna().sum()

ref_date         0
article_url      0
article_title    0
article_date     0
user_url         0
user_name        0
reading_time     0
n_claps          0
dtype: int64

In [34]:
df_articles_info.to_csv("articles_20230103.csv", sep ='\t', index=False)

In [35]:
df_articles_info["n_claps"].describe()

count    6617.000000
mean      129.815778
std       190.300148
min         0.000000
25%        31.000000
50%        75.000000
75%       155.000000
max      3100.000000
Name: n_claps, dtype: float64